In [2]:
##### Importando Bibliotecas: #####
import pandas as pd
import numpy as np
import itertools
import requests
import math
import os
import re

___
# Ciência dos Dados - PROJETO 3 - INSPER

___
### Mariana Abrantes, Gabriel Couto e Gabriel Miras
___

# Previsão de preços de smartphones ao dólar

___

## A. INTRODUÇÃO

Nestes últimos anos, com a instabilidade política e socioeconômica do Brasil, o preço do dólar está em constante variação. Consequentemente, os valores sobre produtos estrangeiros também mudam. Ao visualizar esse cenário, gostaríamos de analisar essa influência da alteração do valor do dólar sobre produtos da área tecnológica, em nosso caso smartphones. 

___
## B. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

[Detalhe aqui as características da base de dados além da análise descritiva.]

__I) Preço do Dólar__ <br>
&emsp;O DataFrame foi obtido no [site](https://economia.uol.com.br/cotacoes/cambio/dolar-comercial-estados-unidos/) e foi filtrado da seguinte forma:
 * a) Definimos a coluna 'Data' como do tipo datetime e em seguida definimos essa coluna como índice da tabela
 * b) Pegamos apenas a coluna 'Compra' com o preço do dolar no dia e o índice das linhas
 * c) Definimos a coluna 'Compra' do tipo float e renomeamos ela pra "Dolar"

In [3]:
dolar = pd.read_table("dolar.csv", sep=";", usecols=['Data','Compra'])
dolar['Data'] = pd.to_datetime(dolar['Data'],  format='%d/%m/%Y %H:%M:%S')
dolar.index = dolar['Data']
dolar['Dolar'] = dolar['Compra'].astype('float')
dolar = dolar.loc[:,['Dolar']]

__II) Tabela de Preços dos Smartphones__ <br>
Os dados de cada celular foram retirados do histórico de preço médio do site buscape. <br>
* a) A função gerarBD() recebe uma lista com nome e url do celular para retornar um Dataframe de preço dos celulares e do dolar por dia.<br>
Para cada linha da lista:
    * 1) Cria uma cópia do Dataframe dolar chamado table
    * 2) Baixa o código da url fornecida e captura o ID do produto
    * 3) Com o ID baixamos o histórico de preço médio do aparelho em JSON e criamos o Dataframe bd com ele
    * 4) Usamos .join para unir as colunas num novo dataframe
    * 5) Definimos a coluna 'Data' como do tipo datetime e em seguida definimos essa coluna como índice da tabela<br>
* b) Removemos as linhas que possuam ao menos um valor vazio e retornamos essa nova tabela como resultado da função

In [5]:
#Recebe uma URL e retorna o código HTML da página (Só é chamada pela função baixarBD)
def baixarhtml(url):
    html = requests.get(url).text.splitlines()
    return html

#Recebe url do Buscape e retorna um Dataframe 
def gerarBD(lista):
    table = dolar.copy()
    
    for num in range(len(lista)):
        produto, url = lista[num][0], lista[num][1]
        
        #Pega ID do produto do buscape usando a url fornecida
        code = baixarhtml(url)
        for num_linha in range(len(code)):
            match = re.findall('"productId":[0-9]*', code[num_linha])
            if len(match)>0:
                id = match[0].split(':')[1]

        #Baixa BD dos preços
        code = baixarhtml("https://www.buscape.com.br/async/history/idu/" + id + "/days/365")

        #Pega só a parte dos dados e cria tabela bd
        bd = pd.read_json(code[0].split('"historicos":')[1][:-1])

        #Converte data pro formato e define essa coluna como índice
        bd['data'] = pd.to_datetime(bd['data'],  format='%d/%m/%Y')
        bd.set_index(bd['data'], inplace=True)
        
        #Adiciona a coluna com o nome do produto o preço do produto
        table[produto] = bd.loc[:,['precomed']]
        table[produto] = table[produto].astype(float)
    
    table = table.dropna(how='any') # Tira linha que tiver algum valor NaN
    return table

#Função retorno
def func_retorno(n,n_ant):
    val = 0
    if n_ant == 0:
        print('tem zero')
    else:
        val = math.log(n/n_ant)
    return val

In [6]:
links = [
    ['Iphone X',          "https://www.buscape.com.br/smartphone-apple-iphone-x-64gb"],
    ['Iphone 8 Plus',     "https://www.buscape.com.br/smartphone-apple-iphone-8-plus-64gb"],
    ['Iphone 7',          "https://www.buscape.com.br/smartphone-apple-iphone-7-32gb"],
    ['Iphone 7 Plus',     "https://www.buscape.com.br/smartphone-apple-iphone-7-plus-128gb"],
    ['Galaxy S8',         "https://www.buscape.com.br/smartphone-samsung-galaxy-s8-sm-g950-64gb"],
    ['Galaxy S8 Plus',    "https://www.buscape.com.br/smartphone-samsung-galaxy-s8-plus-sm-g955-128gb"],
    ['Galaxy S9',         "https://www.buscape.com.br/smartphone-samsung-galaxy-s9-sm-g960-128gb"],
    ['Galaxy S9 Plus',    "https://www.buscape.com.br/smartphone-samsung-galaxy-s9-plus-sm-g965-128gb"]
]

### BASE DE DADOS

Se não houver o arquivo tabela.csv criado executa a função gerarBD() e salva o dataframe como tabela.csv, se já houver o arquivo tabela.csv carrega ele como tabela

In [7]:
#Se já tiver um arquivo tabela.csv salvo carrega ele, se não cria a tabela e salva ela
exists = os.path.isfile('tabela.csv')
if exists:
    tabela = pd.read_csv('tabela.csv', sep=",")
    tabela.index = tabela['data']
    tabela = tabela.loc[:,list(tabela.columns)[1:]]
else:
    tabela = gerarBD(links)
    tabela.to_csv('tabela.csv')

In [8]:
tabela.head()

,Iphone X,Iphone 8 Plus,Iphone 7,Iphone 7 Plus,Galaxy S8,Galaxy S8 Plus,Galaxy S9,Galaxy S9 Plus,Dolar
data,,,,,,,,,
2018-05-22,5791.4575,4019.5854,3033.4468,3491.2202,3050.4950,4085.785,3956.2925,4479.7305,3.6447
2018-05-23,5853.7800,4057.0140,3043.2737,3406.8965,3061.1514,3672.570,3966.1755,4490.3880,3.6251
2018-05-24,5885.4900,3896.7625,2852.5547,3444.2380,2992.7668,4028.438,3966.7402,4502.2856,3.6483
2018-05-25,5876.5710,4019.7207,2985.7068,3470.2570,3130.3193,3685.785,4007.9338,4469.3877,3.6683
2018-05-28,5907.1025,3886.1116,2995.6414,3488.4626,3062.2104,4028.438,3995.7700,4526.9487,3.7286


__III) Criando tabela de retorno__

In [9]:
df = tabela.copy()

#Cria coluna com o valor da linha anterior
for coluna in tabela.columns:
    df[coluna+'-1'] = df[coluna].shift(1)

df = df.iloc[1:,:] # Tira primeira linha (perdida pra fazer deltas)

dic={}
for coluna in tabela.columns:
    lis,ind=[],[]
    for index, row in df.iterrows():
        retor = func_retorno(row[coluna],row[coluna+'-1'])
        lis.append(retor)
        ind.append(index)
    dic[coluna]=lis

retorno = pd.DataFrame(data=dic,index=ind)

In [10]:
retorno.head()

,Iphone X,Iphone 8 Plus,Iphone 7,Iphone 7 Plus,Galaxy S8,Galaxy S8 Plus,Galaxy S9,Galaxy S9 Plus,Dolar
2018-05-23,0.010704,0.009268,0.003234,-0.024450,0.003487,-0.106622,0.002495,0.002376,-0.005392
2018-05-24,0.005402,-0.040301,-0.064719,0.010901,-0.022593,0.092487,0.000142,0.002646,0.006379
2018-05-25,-0.001517,0.031066,0.045622,0.007526,0.044937,-0.088895,0.010331,-0.007334,0.005467
2018-05-28,0.005182,-0.033803,0.003322,0.005232,-0.021998,0.088895,-0.003040,0.012797,0.016304
2018-05-29,0.009240,0.016236,-0.037699,0.005742,0.020046,0.011541,0.001311,0.007073,0.002839


### ANÁLISE DESCRITIVA

Em nossa base de dados buscamos os preços de telefones das marcas Apple e Samsung, sendo eles Iphone X, Iphone 8 Plus, Iphone 7 plus, iphone 7, Galaxy S8, Galaxy S8 Plus, Galaxy S9 e Galaxy S9 Plus. Além disso, pegamos também as características dos modelos escolhidos para que nossa análise seja mais específica, pois ao mudar o tamanho da memória do celular, por exemplo, o preço se altera também. Desse modo, conseguiremos analisar qual a interferência da alteração do dólar nos valores dos smartphones. 

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

[Esse item depende dos resultados das modelagens anteriores! Organize-os aqui de forma clara!]

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS

https://br.investing.com/ $\rightarrow$ site para busca da cotação dos dolores
_________
https://www.buscape.com.br $\rightarrow$ site para busca de histórico dos preços